In [ ]:
# Google Colab specific code for mounting Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define the directory path on your Google Drive
# Replace 'Your_directory' with the actual directory
directory = '/content/drive/My Drive/Your_directory'

In [ ]:
import os

In [ ]:
# Change the working directory to the desired path
os.chdir(directory)

# Verify that the working directory has been changed
print("Current working directory:", os.getcwd())

In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow_probability as tfp

# Load the sorted data directly from the modified CSV
df_sorted = pd.read_csv('Sorted_Encoded_Padded_Probabilities.csv')

def prepare_data(df_part):
    # Extracting features and labels
    X = df_part['Padded'].apply(lambda x: [int(xi) for xi in x.strip('[]').split()]).to_list()
    y = df_part[['Prob1', 'Prob2']].values

    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)

    return X, y

In [ ]:
def map_integers_to_gates(int_sequences, gate_matrices):
    """
    Map each integer in the sequences to its corresponding quantum gate matrix.
    This version also maps zeros to a 4x4 zero matrix.
    
    Args:
    - int_sequences (np.array): An array of shape (batch_size, seq_length) containing integer sequences.
    - gate_matrices (list): A list of 4x4 matrices representing quantum gates.
    
    Returns:
    - np.array: An array of shape (batch_size, seq_length, 4, 4) containing the mapped sequences.
    """
    # Initialize an empty array to store the mapped sequences
    mapped_sequences = np.zeros((int_sequences.shape[0], int_sequences.shape[1], 4, 4))
    
    # Loop through each sequence
    for i in range(int_sequences.shape[0]):
        # Map the integers to their corresponding quantum gate matrices
        for j in range(int_sequences.shape[1]):
            mapped_sequences[i, j] = gate_matrices[int_sequences[i, j]]
            
    return mapped_sequences

In [ ]:
X, y = prepare_data(df_sorted)

print('X type: ', type(X))
print('y type: ', type(y))

# Create new input data
X_new = [X, y]

X_train_0 = X
y_train = y 

# Run the mapping function to convert integer sequences to gate matrices
X_train_mapped = map_integers_to_gates(X_train_0, gate_matrices)
# New input data after mapping
X_train = [X_train_mapped, y_train]

In [ ]:
# Parameters (adjustable)
num_layers = 4  # Number of Transformer layers
num_heads = 4  # Number of attention heads
d_model = 8  # Feature dimension
dff = 512  # Dimension of feed-forward network
group_size = 30  # Example group size
input_length = len(X_train[0][0])  # Example input length
vocab_size = 4

# Positional Encoding Layer (Serializable)
@tf.keras.utils.register_keras_serializable()
class PositionalEncodingLayer(layers.Layer):
    def __init__(self, d_model, **kwargs):
        super(PositionalEncodingLayer, self).__init__(**kwargs)
        self.d_model = d_model

    def build(self, input_shape):
        # Calculate positional encoding for the combined group_size and input_length
        seq_len = input_shape[1] * input_shape[2]  # group_size * input_length
        self.pos_encoding = self.get_positional_encoding(seq_len, self.d_model)

    def call(self, inputs):
        # Reshape inputs to (batch_size, group_size * input_length, d_model) for adding positional encoding
        seq_len = tf.shape(inputs)[1] * tf.shape(inputs)[2]
        reshaped_inputs = tf.reshape(inputs, [-1, seq_len, self.d_model])
        output = reshaped_inputs + self.pos_encoding
        return tf.reshape(output, [-1, tf.shape(inputs)[1], tf.shape(inputs)[2], self.d_model])

    def get_positional_encoding(self, seq_len, d_model):
        angles = np.arange(seq_len)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model))
        sines = np.sin(angles[:, 0::2])
        cosines = np.cos(angles[:, 1::2])

        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_config(self):
        config = super(PositionalEncodingLayer, self).get_config()
        config.update({"d_model": self.d_model})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


# Transformer Encoder Layer (Serializable)
@tf.keras.utils.register_keras_serializable()
class TransformerEncoderLayer(models.Model):
    def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerEncoderLayer, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate

        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dropout1 = layers.Dropout(rate)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)

        self.ffn = models.Sequential([
            layers.Dense(dff, activation='gelu'),
            layers.Dense(d_model)
        ])
        self.dropout2 = layers.Dropout(rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.norm2(out1 + ffn_output)

    def get_config(self):
        config = super(TransformerEncoderLayer, self).get_config()
        config.update({
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config

# Transformer Encoder (Serializable)
@tf.keras.utils.register_keras_serializable()
class TransformerEncoder(models.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_layers = num_layers
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate

        self.enc_layers = [TransformerEncoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]

    def call(self, inputs):
        x = inputs
        for layer in self.enc_layers:
            x = layer(x)
        return x

    def get_config(self):
        config = super(TransformerEncoder, self).get_config()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


# Inputs
input_y_transformer = layers.Input(shape=(group_size, 2), name='input_y_transformer')
input_gate_transformer = layers.Input(shape=(group_size, input_length), name='input_gate_transformer')


# Embedding and Positional Encoding for gate sequences
gate_embedding_layer = layers.Embedding(output_dim=d_model, input_dim=vocab_size)
gate_embeddings = gate_embedding_layer(input_gate_transformer)
gate_pos_encoding_layer = PositionalEncodingLayer(d_model=d_model)
gate_pos_encoding = gate_pos_encoding_layer(gate_embeddings)


# Embedding-like transformation for probabilities
prob_transform_layer = layers.Dense(2 * d_model, activation='gelu')
prob_transformed = prob_transform_layer(input_y_transformer)

# Reshape to match the shape of gate sequence embeddings
prob_reshaped = layers.Reshape((group_size, 2, d_model))(prob_transformed)
prob_pos_encoding_layer = PositionalEncodingLayer(d_model=d_model)
prob_pos_encoding = prob_pos_encoding_layer(prob_reshaped)

# Cross-Attention between Dataset 1 and Dataset 2
cross_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
cross_attention_output = cross_attention(gate_pos_encoding, prob_pos_encoding)

# Applying Transformer Encoder to the output of cross-attention
transformer_block = TransformerEncoder(num_layers, d_model, num_heads, dff)
encoded_output = transformer_block(cross_attention_output)

# Regression Prediction
flattened = layers.Flatten()(encoded_output)
dense_layer = layers.Dense(2048, activation='gelu')(flattened)
dense_layer = layers.Dropout(0.2)(dense_layer)
dense_layer = layers.Dense(1024, activation='gelu')(dense_layer)
dense_layer = layers.Dropout(0.2)(dense_layer)


depolar_error_pred = layers.Dense(256, activation='gelu')(dense_layer)
depolar_error_pred = layers.Dropout(0.2)(depolar_error_pred)
depolar_error_pred = layers.Dense(128, activation='gelu')(depolar_error_pred)
depolar_error_pred = layers.Dense(2, activation='tanh')(depolar_error_pred)


over_rotation_pred = layers.Dense(256, activation='gelu')(dense_layer)
over_rotation_pred = layers.Dropout(0.2)(over_rotation_pred)
over_rotation_pred = layers.Dense(128, activation='gelu')(over_rotation_pred)
over_rotation_pred = layers.Dense(2, activation='tanh')(over_rotation_pred)

output = layers.Concatenate()([depolar_error_pred, over_rotation_pred])

# Full Model
model = models.Model(inputs=[input_gate_transformer, input_y_transformer], outputs=output)


initial_learning_rate = 1e-6
decay_steps = 200
decay_rate = 0.95

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the Model
model.compile(optimizer=optimizer, loss='mse')
model.summary()


In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def easy_PTM_depol_channel(depol_mat):
    identity = tf.eye(4, dtype=tf.float32)
    PTM_depol = (1 - tf.math.abs(depol_mat)) * identity
    
    # Update the value at index [0,0] to 1
    # We use Tensorflow operations to ensure gradient computation
    PTM_depol = tf.tensor_scatter_nd_update(PTM_depol, [[0,0]], [1])
    PTM_depol = tf.cast(PTM_depol, dtype=tf.complex64)
    return PTM_depol



def pauli_matrices():
    """Return the Pauli matrices including identity."""
    I = tf.eye(2, dtype=tf.complex64)
    X = tf.constant([[0, 1], [1, 0]], dtype=tf.complex64)
    Y_imag = tf.constant([[0, -1], [1, 0]], dtype=tf.float32)
    Y_real = tf.constant([[0, 0], [0, 0]], dtype=tf.float32)
    Y = tf.complex(Y_real, Y_imag)
    Z = tf.constant([[1, 0], [0, -1]], dtype=tf.complex64)
    
    return [I, X, Y, Z]

def compute_ideal_ptm(unitary):
    """Compute the ideal PTM from a given unitary."""
    paulis = pauli_matrices()
    ptm_ideal = tf.zeros((4, 4), dtype=tf.complex64)

    for i in range(4):
        for j in range(4):
            term = tf.matmul(unitary, tf.matmul(paulis[j], tf.linalg.adjoint(unitary)))
            trace_value = 0.5 * tf.linalg.trace(tf.matmul(paulis[i], term))
            
            # Update ptm_ideal at position [i, j] with the calculated trace_value
            indices = tf.constant([[i, j]])
            ptm_ideal = tf.tensor_scatter_nd_add(ptm_ideal, indices, [trace_value])
            
    return ptm_ideal


def general_custom_gate(theta, delta, depol_amt, gate):
    # Compute real and imaginary parts as real numbers initially
    real_part = tf.cos((theta + delta) / 2)
    imag_part = tf.sin((theta + delta) / 2)
    
    # Cast them to complex numbers only when necessary
    unitary_rx_adjusted = tf.cast(real_part, dtype=tf.complex64) * tf.eye(2, dtype=tf.complex64) - 1j * tf.cast(imag_part, dtype=tf.complex64) * pauli_matrices()[gate]
    
    ptm_adjusted_rx = compute_ideal_ptm(unitary_rx_adjusted)
    ptm = tf.matmul(easy_PTM_depol_channel(depol_amt), ptm_adjusted_rx)
    
    return tf.math.real(ptm)


In [ ]:
def custom_X(depol_amt, over_rotation):
    return general_custom_gate(theta=math.pi/2, delta=over_rotation, depol_amt=depol_amt, gate=1)

def custom_Y(depol_amt, over_rotation):
    return general_custom_gate(theta=math.pi/2, delta=over_rotation, depol_amt=depol_amt, gate=2)


def custom_gate_set(depol_amt_X, over_rotation_X, depol_amt_Y, over_rotation_Y):
    # Define gates in PTM form

    I = tf.constant([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=tf.float32)  #Gi

    X_theta = custom_X(depol_amt_X, over_rotation_X)
    Y_theta = custom_Y(depol_amt_Y, over_rotation_Y)
    return [X_theta, Y_theta, I]
  
# Define gate application function
def apply_gate(state, gate_set, label):   
    X_theta = gate_set[0]
    Y_theta = gate_set[1]
    I = gate_set[2]
      
    if label == 1:
        return tf.linalg.matmul(X_theta, tf.reshape(state, [-1, 1]))
    elif label == 2:
        return tf.linalg.matmul(Y_theta, tf.reshape(state, [-1, 1]))
    elif label == 3:
        return tf.linalg.matmul(I, tf.reshape(state, [-1, 1]))
    else:
        return state  # If label is 0, don't apply any gate  


def apply_gate_sequence(grouped_gate_sequence, grouped_y_label, grouped_gate_matrix_sequence, grouped_model_output):
    # Initialize a list to collect final states for each data point in the group
    grouped_final_states = []
       
    # Iterate through each data point within the group
    for i in range(tf.shape(grouped_gate_sequence)[0]):
        # print(f"current gate sequence number: {i}")
        single_gate_sequence = tf.gather(grouped_gate_sequence, i, axis=0)
        
        single_y_label = tf.gather(grouped_y_label, i, axis=0)
        single_gate_matrix_sequence = tf.gather(grouped_gate_matrix_sequence, i, axis=0)
        
        # Initialize state in Pauli basis
        state = tf.constant([1/math.sqrt(2), 0, 0, 1/math.sqrt(2)], dtype=tf.float32)
        
        # Extract the model output corresponding to this data point
        depol_amt_X, depol_amt_Y, over_rotation_X, over_rotation_Y = grouped_model_output
       
        # Reconstruct the gate set using the predicted depolarization and over-rotation
        reconstructed_gate_set = custom_gate_set(depol_amt_X, over_rotation_X, depol_amt_Y, over_rotation_Y)
        
        

        # Apply each gate in the sequence
        debug_label_arr = []
        for j in range(tf.shape(single_gate_sequence)[0]):
            debug_label_arr.append(single_gate_sequence[j])
            if single_gate_sequence[j] == 0:
                break
            state = apply_gate(state, reconstructed_gate_set, single_gate_sequence[j])

        # Append the final state for this data point to the list of final states
        grouped_final_states.append(state)

    # Convert the list of final states into a tensor
    grouped_final_states = tf.stack(grouped_final_states)   
    return grouped_final_states


def compute_probabilities(grouped_ptm_vector):    
    # Remove singleton dimensions if any (like the last '1' in [10, 4, 1])
    grouped_ptm_vector = tf.squeeze(grouped_ptm_vector, axis=-1)    
    ptm_0 = tf.constant([1/math.sqrt(2), 0, 0, 1/math.sqrt(2)], dtype=tf.float32)
    ptm_1 = tf.constant([1/math.sqrt(2), 0, 0, -1/math.sqrt(2)], dtype=tf.float32)
    
    # Compute dot products in a batched manner using broadcasting
    prob_0 = tf.reduce_sum(grouped_ptm_vector * ptm_0, axis=-1)
    prob_1 = tf.reduce_sum(grouped_ptm_vector * ptm_1, axis=-1)

    final_probabilities = tf.stack([prob_0, prob_1], axis=-1)
    
    return final_probabilities



# Define loss function
loss_fn = MeanSquaredError()
  



def train_step(X, y, X_ms):
    train_losses = []  # Initialize a list to collect the losses for each group in the mini-batch
    
    with tf.GradientTape() as tape:
        # Compute model output for the entire mini-batch here
        full_batch_model_output = model([X_ms[0], X[1]])
        
        # Loop over each group in the mini-batch
        for i in range(tf.shape(X[0])[0]):
            single_group_gate_sequence = tf.gather(X[0], i, axis=0)
            single_group_y_label = tf.gather(X[1], i, axis=0)
            single_group_gate_matrix_sequence = tf.gather(X_ms[0], i, axis=0)

            # Gather the model's output for the i-th group from the full batch output
            single_group_model_output = tf.gather(full_batch_model_output, i, axis=0)

            # Process a single group
            final_states = apply_gate_sequence(single_group_gate_sequence, single_group_y_label, single_group_gate_matrix_sequence, single_group_model_output)
            probabilities = compute_probabilities(final_states)
            
            # Calculate loss for this group
            loss = loss_fn(tf.gather(y, i, axis=0), probabilities)
            train_losses.append(loss)
            
        # Average the losses across the mini-batch
        average_loss = tf.reduce_mean(train_losses)

    # Compute the gradients for the entire mini-batch and update model parameters
    grads = tape.gradient(average_loss, model.trainable_weights)
       
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    return average_loss, tf.squeeze(full_batch_model_output)

In [ ]:
# Directory to save the model and weights
model_save_dir = "saved_models"
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
def reshape_for_grouping(data, group_size):
    """
    Reshape the data for grouping with repeating the last indices in case of remainder.
    
    Parameters:
        data (numpy.ndarray): The data to be reshaped.
        group_size (int): The desired group size.
        
    Returns:
        numpy.ndarray: The reshaped data.
    """
    n_samples = data.shape[0]
    n_batches = n_samples // group_size
    remainder = n_samples % group_size
    
    # If the remainder exists, pad the data by repeating the last indices
    if remainder:
        padding_count = group_size - remainder
        padding_indices = [-i % n_samples - 1 for i in range(padding_count)]
        padding = data[padding_indices]
        data = np.concatenate([data, padding], axis=0)
        
    return data.reshape(-1, group_size, *data.shape[1:])

X_train_0_grouped = reshape_for_grouping(X_train_0, group_size)
y_train_grouped = reshape_for_grouping(y_train, group_size)
X_train_grouped = [reshape_for_grouping(X, group_size) for X in X_train]

In [ ]:
# Training loop
EPOCHS = 100
num_parts = 1 # num_parts = 1 means no curriculum learning
part_size = len(X_train_0_grouped) // num_parts
print(f"Length of X_train_0_grouped: {len(X_train_0_grouped)}")
print(f"part size: {part_size}")

# Lists to store the mean train and validation losses for each epoch
all_train_losses = []
all_train_predicted_values = []
all_train_X_depolar = []
all_train_Y_depolar = []
all_train_predicted_X_overrotation = []
all_train_predicted_Y_overrotation = []


# List to store learning rates
learning_rates = []

total_epochs_elapsed = 0  # Counter for the total number of epochs elapsed

end_idx = 0  # Initialize end_idx to zero


for part in range(num_parts):
    # Determine the dataset subset for the current part
    start_idx = part * part_size
    end_idx = (part + 1) * part_size  # Exclusive
    
    print(f"start_idx: {start_idx}")
    print(f"end_idx: {end_idx}")
    

    X_subset = X_train_0_grouped[start_idx:end_idx]
    print(f"Shape of X_subset: {X_subset.shape}")

    y_subset = y_train_grouped[start_idx:end_idx]
    print(f"Shape of y_subset: {y_subset.shape}")

    X_subset_ms = [X_train_grouped[0][start_idx:end_idx], X_train_grouped[1][start_idx:end_idx]]

    for epoch in range(EPOCHS):
        train_losses_per_epoch = []
        predicted_values_per_epoch = np.zeros(4) # Initialize zeros array, number of zeros corresponds to model output dimension 
        predicted_values_counter_per_epoch = 0 

        for i in range(len(X_subset)):
            X_batch = [X_subset[i:i + 1], y_subset[i:i + 1]]
            X_batch_ms = [X_subset_ms[0][i:i + 1], X_subset_ms[1][i:i + 1]]
            y_batch = y_subset[i:i + 1]

            train_loss, train_predicted_values = train_step(X_batch, y_batch, X_batch)
                        
            train_losses_per_epoch.append(train_loss)
            predicted_values_per_epoch = predicted_values_per_epoch + np.array(train_predicted_values)
            predicted_values_counter_per_epoch += 1
                      

        mean_train_loss = np.mean(train_losses_per_epoch)
        mean_predicted_values = predicted_values_per_epoch/predicted_values_counter_per_epoch

        # Store the mean losses for this epoch
        all_train_losses.append(mean_train_loss)
        all_train_predicted_values.append(mean_predicted_values)
        
        all_train_X_depolar.append(mean_predicted_values[0])
        all_train_Y_depolar.append(mean_predicted_values[1])
        all_train_predicted_X_overrotation.append(mean_predicted_values[2])
        all_train_predicted_Y_overrotation.append(mean_predicted_values[3])
            

        # Record the learning rate at this epoch
        current_step = model.optimizer.iterations
        current_lr = lr_schedule(current_step).numpy()
        learning_rates.append(current_lr)

        total_epochs_elapsed += 1
        print(f"Part: {part+1}/{num_parts}, Epoch: {epoch+1}/{EPOCHS}, Total Epochs: {total_epochs_elapsed}, Train Loss: {mean_train_loss}, Learning Rate: {current_lr}")
        if (total_epochs_elapsed % 10 == 0) or (total_epochs_elapsed == 1) or (epoch+1 == EPOCHS):
            model_path = os.path.join(model_save_dir, f"model_epoch_{total_epochs_elapsed}")
            weights_path = os.path.join(model_save_dir, f"weights_epoch_{total_epochs_elapsed}")
            if (total_epochs_elapsed % 100 == 0) or (epoch+1 == EPOCHS):
                model.save(f"{model_path}.keras", save_format='keras')
                print(f"Model and weights saved at epoch {total_epochs_elapsed}")          

            loss_lr_df = pd.DataFrame({
                'Epoch': list(range(1, len(all_train_losses) + 1)),
                'Training_Loss': all_train_losses,
                'Learning_Rate': learning_rates # Include learning rates
            })
            
            predicted_values_df = pd.DataFrame({
                'Epoch': list(range(1, len(all_train_predicted_values) + 1)),
                'Predicted_Values': all_train_predicted_values,
                'X_Depolarizing_Error': all_train_X_depolar,
                'Y_Depolarizing_Error': all_train_Y_depolar,
                'X_Over_Rotation_Error': all_train_predicted_X_overrotation,
                'Y_Over_Rotation_Error':all_train_predicted_Y_overrotation
            })
            
            # Save the DataFrame to a CSV file
            loss_lr_df.to_csv(f'losses_and_lr_epoch_{total_epochs_elapsed}.csv', index=False)
            predicted_values_df.to_csv(f'predicted_values_epoch_{total_epochs_elapsed}.csv', index=False)

            print(f"Losses and learning rates saved at epoch {total_epochs_elapsed}")
            

        if (part+1) != num_parts:
            if mean_train_loss <= 2.5e-5:
                print(f"mean_train_loss: {mean_train_loss} <= 2.5e-5, skip to next stage.")
                break
                